<a href="https://colab.research.google.com/github/GiorgioMB/UniversityProjects/blob/main/Machine%20Learning%20Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**:
- Unless you have a GPU, some blocks will be slow
- You will be prompted to load your Google Drive, allow it (it expects you have the zipped file from blackboard loaded on it)



In [1]:
!pip install autoviz
!pip install imbalanced-learn
!pip install optuna
!pip install shap
!git clone https://github.com/GiorgioMB/SPORF_Available.git
##Sparse Projection Oblique Randomer Forests, I just fixed it,
##I don't want to publish it on pypi because technically it isn't mine
##If you're wondering what was the problem, it was a C++ header that was used for the
##unsupervised version, which we don't use for obvious reasons, that was deprecated sometime in
##The last 8 years.

  Using cached optuna-3.6.1-py3-none-any.whl (380 kB)
  Using cached alembic-1.13.1-py3-none-any.whl (233 kB)
  Using cached colorlog-6.8.2-py3-none-any.whl (11 kB)
  Using cached Mako-1.3.3-py3-none-any.whl (78 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 7.0 MB/s eta 0:00:00
Cloning into 'SPORF_Available'...
remote: Enumerating objects: 6696, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 6696 (delta 4), reused 0 (delta 0), pack-reused 6682
Receiving objects: 100% (6696/6696), 46.21 MiB | 11.09 MiB/s, done.
Resolving deltas: 100% (3573/3573), done.
Updating files: 100% (2112/2112), done.


In [2]:
!python /content/SPORF_Available/Python/setup.py clean --all
!pip install /content/SPORF_Available/Python
##This is just to finish the installation process

/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 6.1 MB/s eta 0:00:00
running clean
'build/lib.linux-x86_64-cpython-310' does not exist -- can't clean it
'build/bdist.linux-x86_64' does not exist -- can't clean it
'build/scripts-3.10' does not exist -- can't clean it
Processing ./SPORF_Available/Python
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for rerf: filename=rerf-2.0.5-cp310-cp310-linux_x86_64.wh


# Data Preprocessing

We start by importing the relevant libraries and the files necessary

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from rerf.rerfClassifier import rerfClassifier ##May not work if there were errors in the installation process
from autoviz.AutoViz_Class import AutoViz_Class ## Works only on GPU accelerated notebooks, do I know why? No
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler, Normalizer
from xgboost import XGBClassifier
import optuna as opt
import zipfile
import statsmodels.api as sm
import statsmodels.formula.api as smf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LogisticRegression
import os
rand_val = 42 ##For reproducibility, you can change this to any value you like
np.random.seed(rand_val)
torch.manual_seed(rand_val)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
window_user = False ##Change this if you're on a Windows Device

Imported v0.1.904. Please call AutoViz in this sequence:
    AV = AutoViz_Class()
    %matplotlib inline
    dfte = AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=1, lowess=False,
               chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30, save_plot_dir=None)


Assuming that you have `data.zip` in your Google Drive folder and that you enabled Drive Mounting for this notebook



In [2]:
from google.colab import drive
drive.mount('/content/drive') ##Grant it permission so it can read the train.csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
zip_path = '/content/drive/MyDrive/data.zip'
extract_to = '/content/' ##Unzips it
os.makedirs(extract_to, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
print("Extraction complete!")

Extraction complete!


In [4]:
if window_user: df = pd.read_csv('/content/train.csv', index_col = 0) ##For windows user
else: df = pd.read_csv('/content/data/train.csv', index_col = 0)

## Data Exploration

In [5]:
AV = AutoViz_Class()
filename = ""
sep = ","
dft = AV.AutoViz( ##This is just to get a table that checks whether our data has outliers and/or has correlation
    filename,
    sep=sep,
    dfte=df,
    depVar="",
    chart_format="svg",
    max_rows_analyzed=1112499, ##The number of rows in our dataset
    max_cols_analyzed=11 ##The number of columns
)

Shape of your Data Set loaded: (112500, 11)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    Number of Numeric Columns =  10
    Number of Integer-Categorical Columns =  0
    Number of String-Categorical Columns =  0
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  0
    Number of Numeric-Boolean Columns =  1
    Number of Discrete String Columns =  0
    Number of NLP String Columns =  0
    Number of Date Time Columns =  0
    Number of ID Columns =  0
    Number of Columns to Delete =  0
    11 Predictors classified...
        No variables removed since no ID or low-information variables found in data set
10 numeric variables in data exceeds limit, taking top 11 variables
    List of variables 

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
SeriousDlqin2yrs,float64,0.000000,0,0.000000,1.000000,No issue
RevolvingUtilizationOfUnsecuredLines,float64,0.000000,NA,0.000000,50708.000000,Column has 595 outliers greater than upper bound (1.35) or lower than lower bound(-0.76). Cap them or remove them.
age,float64,0.000000,NA,-1.000000,103.000000,Column has 37 outliers greater than upper bound (96.00) or lower than lower bound(8.00). Cap them or remove them.
NumberOfTime30-59DaysPastDueNotWorse,float64,0.000000,NA,0.000000,98.000000,Column has 17994 outliers greater than upper bound (0.00) or lower than lower bound(0.00). Cap them or remove them.
DebtRatio,float64,0.000000,NA,0.000000,329664.000000,Column has 23330 outliers greater than upper bound (1.89) or lower than lower bound(-0.85). Cap them or remove them.
MonthlyIncome,float64,19.479790,NA,0.000000,2866005.615861,"21846 missing values. Impute them with mean, median, mode, or a constant value such as 123., Column has 3741 outliers greater than upper bound (14690.31) or lower than lower bound(-3649.00). Cap them or remove them."
NumberOfOpenCreditLinesAndLoans,float64,0.000000,NA,0.000000,58.000000,Column has 2986 outliers greater than upper bound (20.00) or lower than lower bound(-4.00). Cap them or remove them.
NumberOfTimes90DaysLate,float64,0.000000,NA,0.000000,98.000000,"Column has 6257 outliers greater than upper bound (0.00) or lower than lower bound(0.00). Cap them or remove them., Column has a high correlation with ['NumberOfTime30-59DaysPastDueNotWorse']. Consider dropping one of them."
NumberRealEstateLoansOrLines,float64,0.000000,NA,0.000000,54.000000,Column has 595 outliers greater than upper bound (5.00) or lower than lower bound(-3.00). Cap them or remove them.
NumberOfTime60-89DaysPastDueNotWorse,float64,0.000000,NA,0.000000,98.000000,"Column has 5693 outliers greater than upper bound (0.00) or lower than lower bound(0.00). Cap them or remove them., Column has a high correlation with ['NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTimes90DaysLate']. Consider dropping one of them."


Number of All Scatter Plots = 55
No categorical or numeric vars in data set. Hence no bar charts.
All Plots done
Time to run AutoViz = 64 seconds 

 ###################### AUTO VISUALIZATION Completed ########################


Our data is very much unclean and it may have some values that make no sense, for example we have observation(s) where $age = -1$ and data that is highly correlated (which we do not like)

In [6]:
df[df['age'] == -1] ##Check how many rows actually have a -1 (it's one observation)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
5005,0.0,1.0,-1.0,1.0,0.436927,5620.915291,6.0,0.0,2.0,0.0,2.0


In [7]:
df[df['MonthlyIncome'].isna()]['SeriousDlqin2yrs'].value_counts()

SeriousDlqin2yrs
0.0    20958
1.0     1229
Name: count, dtype: int64

In [8]:
df['SeriousDlqin2yrs'].value_counts()

SeriousDlqin2yrs
0.0    104980
1.0      7520
Name: count, dtype: int64

## Data cleaning

We now start by removing the non-sensical values where $age < 0$. We could simply set $age = \text{NaN}$ but since the target variable is $0$, the majority class, for that observation, we simply drop the row

In [9]:
df = df[df['age'] > 0] ##Remove the -1 observation

In [10]:
df.rename(columns = {'NumberOfTime30-59DaysPastDueNotWorse':'NumberOfTime3059DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse': 'NumberOfTime6089DaysPastDueNotWorse'}, inplace = True)
mod1 = smf.ols(' NumberOfTimes90DaysLate ~ NumberOfTime3059DaysPastDueNotWorse', data=df).fit()
residuals = mod1.resid
df['NumberOfTimes90DaysLate'] = residuals ##Take NumberOfTimes90DaysLate as the residual of NumberOfTime3059DaysPastDueNotWorse

mod2 = smf.ols('NumberOfTime6089DaysPastDueNotWorse ~ NumberOfTime3059DaysPastDueNotWorse + NumberOfTimes90DaysLate', data=df).fit()
residuals = mod2.resid
df['NumberOfTime6089DaysPastDueNotWorse'] = residuals ##The same thing as before, we just take it to be the residuals of the other two

In [11]:
##Now we check that there's no correlation anymore
AV = AutoViz_Class()
filename = ""
sep = ","
dft = AV.AutoViz(
    filename,
    sep=sep,
    dfte=df,
    depVar="",
    chart_format="svg",
    max_rows_analyzed=1112499, ##The number of rows in our dataset
    max_cols_analyzed=11 ##The number of columns
)

Shape of your Data Set loaded: (112499, 11)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    Number of Numeric Columns =  10
    Number of Integer-Categorical Columns =  0
    Number of String-Categorical Columns =  0
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  0
    Number of Numeric-Boolean Columns =  1
    Number of Discrete String Columns =  0
    Number of NLP String Columns =  0
    Number of Date Time Columns =  0
    Number of ID Columns =  0
    Number of Columns to Delete =  0
    11 Predictors classified...
        No variables removed since no ID or low-information variables found in data set
10 numeric variables in data exceeds limit, taking top 11 variables
    List of variables 

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
SeriousDlqin2yrs,float64,0.000000,0,0.000000,1.000000,No issue
RevolvingUtilizationOfUnsecuredLines,float64,0.000000,NA,0.000000,50708.000000,Column has 595 outliers greater than upper bound (1.35) or lower than lower bound(-0.76). Cap them or remove them.
age,float64,0.000000,NA,20.000000,103.000000,Column has 36 outliers greater than upper bound (96.00) or lower than lower bound(8.00). Cap them or remove them.
NumberOfTime3059DaysPastDueNotWorse,float64,0.000000,NA,0.000000,98.000000,Column has 17993 outliers greater than upper bound (0.00) or lower than lower bound(0.00). Cap them or remove them.
DebtRatio,float64,0.000000,NA,0.000000,329664.000000,Column has 23330 outliers greater than upper bound (1.89) or lower than lower bound(-0.85). Cap them or remove them.
MonthlyIncome,float64,19.479964,NA,0.000000,2866005.615861,"21846 missing values. Impute them with mean, median, mode, or a constant value such as 123., Column has 3741 outliers greater than upper bound (14690.62) or lower than lower bound(-3649.22). Cap them or remove them."
NumberOfOpenCreditLinesAndLoans,float64,0.000000,NA,0.000000,58.000000,Column has 2986 outliers greater than upper bound (20.00) or lower than lower bound(-4.00). Cap them or remove them.
NumberOfTimes90DaysLate,float64,0.000000,NA,-12.570443,17.145368,Column has 20997 outliers greater than upper bound (0.15) or lower than lower bound(0.15). Cap them or remove them.
NumberRealEstateLoansOrLines,float64,0.000000,NA,0.000000,54.000000,Column has 595 outliers greater than upper bound (5.00) or lower than lower bound(-3.00). Cap them or remove them.
NumberOfTime6089DaysPastDueNotWorse,float64,0.000000,NA,-11.366346,9.727752,Column has 22780 outliers greater than upper bound (0.07) or lower than lower bound(0.07). Cap them or remove them.


Number of All Scatter Plots = 55
No categorical or numeric vars in data set. Hence no bar charts.
All Plots done
Time to run AutoViz = 20 seconds 

 ###################### AUTO VISUALIZATION Completed ########################


Now, as our dataset has a (significant) number of NaNs, we move to use sklearn's `IterativeImputer`. But first, to avoid any kind of data leak, where `IterativeImputer` uses some information from our target variable to impute NaNs, we split our dataset into the feature matrix $X$ and target vector $y$



In [12]:
y = df['SeriousDlqin2yrs']
X = df.drop(columns = 'SeriousDlqin2yrs')

In [13]:
imp = IterativeImputer(max_iter=10, random_state=rand_val)
imp.fit(X)
X_imp = imp.transform(X)

We now check the amount of 0s and 1s within our target vector

In [14]:
y.value_counts()

SeriousDlqin2yrs
0.0    104979
1.0      7520
Name: count, dtype: int64

As our target variable has a very unbalanced number of 0s and 1s, we move on to use Synthetic Minority Oversampling TEchnique (SMOTE)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.25, random_state=rand_val, stratify = y)
smote = SMOTE(random_state=rand_val) ## Create synthetic observations of SeriousDlqin2yrs == 1
##Sidenote: Maybe we could try ADASYN, even though I highly doubt it changes results drammatically
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [16]:
imp.get_params()

{'add_indicator': False,
 'estimator': None,
 'imputation_order': 'ascending',
 'initial_strategy': 'mean',
 'keep_empty_features': False,
 'max_iter': 10,
 'max_value': inf,
 'min_value': -inf,
 'missing_values': nan,
 'n_nearest_features': None,
 'random_state': 42,
 'sample_posterior': False,
 'skip_complete': False,
 'tol': 0.001,
 'verbose': 0}

In [17]:
imp.imputation_sequence_

[_ImputerTriplet(feat_idx=0, neighbor_feat_idx=array([1, 2, 3, 4, 5, 6, 7, 8, 9]), estimator=BayesianRidge()),
 _ImputerTriplet(feat_idx=1, neighbor_feat_idx=array([0, 2, 3, 4, 5, 6, 7, 8, 9]), estimator=BayesianRidge()),
 _ImputerTriplet(feat_idx=2, neighbor_feat_idx=array([0, 1, 3, 4, 5, 6, 7, 8, 9]), estimator=BayesianRidge()),
 _ImputerTriplet(feat_idx=3, neighbor_feat_idx=array([0, 1, 2, 4, 5, 6, 7, 8, 9]), estimator=BayesianRidge()),
 _ImputerTriplet(feat_idx=5, neighbor_feat_idx=array([0, 1, 2, 3, 4, 6, 7, 8, 9]), estimator=BayesianRidge()),
 _ImputerTriplet(feat_idx=6, neighbor_feat_idx=array([0, 1, 2, 3, 4, 5, 7, 8, 9]), estimator=BayesianRidge()),
 _ImputerTriplet(feat_idx=7, neighbor_feat_idx=array([0, 1, 2, 3, 4, 5, 6, 8, 9]), estimator=BayesianRidge()),
 _ImputerTriplet(feat_idx=8, neighbor_feat_idx=array([0, 1, 2, 3, 4, 5, 6, 7, 9]), estimator=BayesianRidge()),
 _ImputerTriplet(feat_idx=9, neighbor_feat_idx=array([0, 1, 2, 3, 4, 5, 6, 7, 8]), estimator=BayesianRidge()),
 

In [18]:
adasyn = ADASYN(random_state = rand_val)
X_train_ada, y_train_ada = adasyn.fit_resample(X_train, y_train)

# Logistic Regression

In [19]:
clf = LogisticRegression(penalty = 'l2', tol = 1e-6, solver = 'newton-cholesky', max_iter = 10000, n_jobs = -1).fit(X_train_smote, y_train_smote)

In [20]:
def evaluate_model(model, inputs, true_vals):
    y_pred = clf.predict(inputs)
    y_probs = clf.predict_proba(inputs)[:, 1]
    roc_auc = roc_auc_score(true_vals, y_probs)
    f1 = f1_score(true_vals, y_pred)
    cm = confusion_matrix(true_vals, y_pred)
    precision = precision_score(true_vals, y_pred)
    recall = recall_score(true_vals, y_pred)
    accuracy = accuracy_score(true_vals, y_pred)
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:\n", cm)
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Accuracy: {accuracy:.4f}")

evaluate_model(clf, X_test, y_test)

ROC AUC Score: 0.8022
F1 Score: 0.3121
Confusion Matrix:
 [[21538  4707]
 [  662  1218]]
Precision: 0.2056
Recall: 0.6479
Accuracy: 0.8091


In [27]:
shap.initjs()
X_test_df = pd.DataFrame(X_test, columns = X.columns)
explainer = shap.LinearExplainer(clf, X_test_df)
shap_values = explainer.shap_values(X_test)
shap_values_exp = shap.Explanation(shap_values, X_test_df)
shap.plots.bar(shap_values_exp) ##THIS DOES NOT WORK ON COLAB
col_to_number = {col: i for i, col in enumerate(X.columns)}
print(col_to_number)

{'RevolvingUtilizationOfUnsecuredLines': 0, 'age': 1, 'NumberOfTime3059DaysPastDueNotWorse': 2, 'DebtRatio': 3, 'MonthlyIncome': 4, 'NumberOfOpenCreditLinesAndLoans': 5, 'NumberOfTimes90DaysLate': 6, 'NumberRealEstateLoansOrLines': 7, 'NumberOfTime6089DaysPastDueNotWorse': 8, 'NumberOfDependents': 9}


# Neural Network

In [22]:
class CreditDataset(Dataset):
    def __init__(self, X, y, transform=None, pre_transform=None):
        super(CreditDataset, self).__init__()
        self.X = pd.DataFrame(X)
        self.y = pd.DataFrame(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X.iloc[idx], self.y.iloc[idx]

def collate_fn(batch):
    # Unzip the batch to separate features and labels
    X, y = zip(*batch)
    X = torch.stack([torch.tensor(x, dtype=torch.float32) for x in X]).to(device)
    y = torch.tensor(y, dtype=torch.int64).to(device)
    return X, y
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train_smote)
X_test_std = scaler.fit_transform(X_test)
train_dataset = CreditDataset(X_train_std, y_train_smote)
val_dataset = CreditDataset(X_test_std, y_test)
train_loader = DataLoader(train_dataset, batch_size=X_train_smote.shape[0], shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=10000, shuffle=False, collate_fn=collate_fn)

In [23]:
class CreditClassifier(nn.Module):
    def __init__(self, input_shape, dropout_rate = 0):
        super(CreditClassifier, self).__init__()
        ##Misc layers
        self.ELU = nn.ELU()
        self.drop = nn.Dropout(dropout_rate)

        ##Linear (or Dense) Layers
        self.linear1 = nn.Linear(input_shape, 64)
        self.linear2 = nn.Linear(64, 32)
        self.linear3 = nn.Linear(32, 1)

        ##Sigmoid function
        self.classifier = nn.Sigmoid()

    def forward(self, x):
        x = self.drop(self.ELU(self.linear1(x)))
        x = self.drop(self.ELU(self.linear2(x)))
        x = self.linear3(x)
        return  self.classifier(x)

model = CreditClassifier(X_train_smote.shape[1], 0.1).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

## Model Training and Evaluation

In [24]:
def train_model(model, train_loader, val_loader, criterion, optimizer, l1 = True, l2 = False, num_epochs=100, l1_lambda = 1e-4, l2_lambda = 1e-4):
    print("Beginning training")
    print("Training a model with", sum(p.numel() for p in model.parameters() if p.requires_grad))
    print("Overview of the model:", model)
    if device.type == 'cpu':
        print("Notice: you are using a CPU, this may take a while...\n Prints for training and validation will be separate")
    if l1:
      print("Notice: you are adding l1 regularization with lambda", l1_lambda)
    if l2:
      print("Notice: you are adding l2 regularization with lambda", l2_lambda)
    for epoch in range(num_epochs):
        model.train() ##Set model to train
        running_loss = 0.0
        all_outputs = []
        all_targets = []
        for data, targets in train_loader:
            optimizer.zero_grad() ##Remove the gradient of the previous batch
            outputs = model(data).to(torch.float).cpu() ##Forward pass
            targets = targets.to(torch.float).cpu()
            loss = criterion(outputs, targets) ##Loss calculation
            if l1:
                l1_reg = sum(param.abs().sum() for param in model.parameters()).cpu() ##L1 Regularization
                loss += l1_lambda * l1_reg
            if l2:
                l2_reg = sum(torch.norm(param)**2 for param in model.parameters()).cpu() ##L2 Regularization
                loss += l2_lambda * l2_reg
            loss.backward() ##Backward pass
            optimizer.step() ##Gradient descent
            running_loss += loss.item() * data.size(0)
            all_outputs.append(outputs)
            all_targets.append(targets)

        ##For print purpouses only
        all_outputs = torch.cat(all_outputs)
        all_targets = torch.cat(all_targets)
        if isinstance(all_outputs, torch.Tensor):
            all_outputs = all_outputs.cpu().detach().numpy()
        if isinstance(all_targets, torch.Tensor):
            all_targets = all_targets.cpu().detach().numpy()
        train_roc = roc_auc_score(all_targets, all_outputs)
        epoch_loss = running_loss / len(train_loader.dataset)
        ##

        if device.type == 'cpu': ##Have an intermediate print if we're running on CPU only
            print(f'\rEpoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Training ROC AUC Score {train_roc:.4f}')

        val_loss = 0.0
        all_outputs = []
        all_targets = []
        model.eval() ##Set model to evaluation
        with torch.no_grad(): ##Don't compute gradients
            for data, targets in val_loader:
                outputs = model(data).to(torch.float).cpu() ##Prediction of y
                targets = targets.to(torch.float).cpu()
                loss = criterion(outputs, targets) ##Calculate loss
                val_loss += loss.item() * data.size(0)
                all_outputs.append(outputs)
                all_targets.append(targets)

            ##For printing purposes only
            val_loss /= len(val_loader.dataset)
            all_outputs = torch.cat(all_outputs)
            all_targets = torch.cat(all_targets)
            if isinstance(all_outputs, torch.Tensor):
                all_outputs = all_outputs.cpu().numpy()
            if isinstance(all_targets, torch.Tensor):
                all_targets = all_targets.cpu().numpy()
            roc = roc_auc_score(all_targets, all_outputs)

        if device.type == 'cpu': ##Second print for the CPU
            print(f' Epoch [{epoch+1}/{num_epochs}],Validation Loss: {val_loss:.4f}, Validation ROC AUC Score: {roc:.4f}\n')
        else: ##Full print for GPU
            print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Training ROC AUC Score {train_roc:.4f}, Validation Loss: {val_loss:.4f}, Validation ROC AUC Score: {roc:.4f}')
    if l1:
      nonzero = sum(p.numel() for p in model.parameters() if p.requires_grad and p.data.sum() != 0)
      og = sum(p.numel() for p in model.parameters() if p.requires_grad)
      print('Regularization removed', og - nonzero, 'parameters')
    print('Training complete')

train_model(model, train_loader, val_loader, criterion, optimizer)

Beginning training
Training a model with 2817
Overview of the model: CreditClassifier(
  (ELU): ELU(alpha=1.0)
  (drop): Dropout(p=0.1, inplace=False)
  (linear1): Linear(in_features=10, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=32, bias=True)
  (linear3): Linear(in_features=32, out_features=1, bias=True)
  (classifier): Sigmoid()
)
Notice: you are adding l1 regularization with lambda 0.0001
Epoch [1/100], Training Loss: 0.7183, Training ROC AUC Score 0.5036, Validation Loss: 0.6768, Validation ROC AUC Score: 0.5896
Epoch [2/100], Training Loss: 0.7130, Training ROC AUC Score 0.5571, Validation Loss: 0.6792, Validation ROC AUC Score: 0.6386
Epoch [3/100], Training Loss: 0.7081, Training ROC AUC Score 0.6114, Validation Loss: 0.6816, Validation ROC AUC Score: 0.6851
Epoch [4/100], Training Loss: 0.7032, Training ROC AUC Score 0.6564, Validation Loss: 0.6839, Validation ROC AUC Score: 0.7063
Epoch [5/100], Training Loss: 0.6986, Training ROC AUC Score 0

In [25]:
def evaluate_model(model, dataloader):
    model.eval() ##Self explanatory
    y_preds, y_probs, true_vals = [], [], []

    with torch.no_grad(): ##Gradient computation not needed
        for inputs, labels in dataloader:
            outputs = model(inputs).squeeze()
            predicted_probs = outputs
            predicted_labels = (predicted_probs >= 0.5).long()
            y_probs.extend(predicted_probs.cpu().numpy())
            y_preds.extend(predicted_labels.cpu().numpy())
            true_vals.extend(labels.cpu().numpy())

    ##Metrics computation
    roc_auc = roc_auc_score(true_vals, y_probs)
    f1 = f1_score(true_vals, y_preds)
    cm = confusion_matrix(true_vals, y_preds)
    precision = precision_score(true_vals, y_preds)
    recall = recall_score(true_vals, y_preds)
    accuracy = accuracy_score(true_vals, y_preds)

    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:\n", cm)
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Accuracy: {accuracy:.4f}")

evaluate_model(model, val_loader)

ROC AUC Score: 0.7874
F1 Score: 0.2130
Confusion Matrix:
 [[15700 10545]
 [  399  1481]]
Precision: 0.1231
Recall: 0.7878
Accuracy: 0.6109


In [26]:
##Make the test set balanced, ie undersample the 0 target variable
mask_class_0 = (y_test == 0)
mask_class_1 = (y_test == 1)
X_test_class_0 = X_test_std[mask_class_0]
X_test_class_1 = X_test_std[mask_class_1]
y_test_class_0 = y_test[mask_class_0].to_numpy()
y_test_class_1 = y_test[mask_class_1].to_numpy()
n_samples_per_class = min(len(y_test_class_0), len(y_test_class_1))

##Take indeces at random and make a test set where the target variable is balanced
indices_class_0 = np.random.choice(range(len(y_test_class_0)), n_samples_per_class, replace=False)
indices_class_1 = np.random.choice(range(len(y_test_class_1)), n_samples_per_class, replace=False)
X_test_balanced = np.concatenate((X_test_class_0[indices_class_0], X_test_class_1[indices_class_1]), axis=0)
y_test_balanced = np.concatenate((y_test_class_0[indices_class_0], y_test_class_1[indices_class_1]), axis=0)

##Create a Dataset and Loader to feed to the model
balanced_dataset = CreditDataset(X_test_balanced, y_test_balanced)
balanced_loader = DataLoader(balanced_dataset, batch_size=1000, shuffle=False, collate_fn=collate_fn)

##Call the evaluation
evaluate_model(model, balanced_loader)

ROC AUC Score: 0.7867
F1 Score: 0.7184
Confusion Matrix:
 [[1118  762]
 [ 399 1481]]
Precision: 0.6603
Recall: 0.7878
Accuracy: 0.6912


# XGBoost

## Model training and hyperparameter optimization

In [28]:
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'subsample': trial.suggest_float('subsample', 0.5, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'objective': 'binary:logistic',
        'n_jobs': -1,
        'tree_method': 'gpu_hist',  ## Use GPU histogram tree method
        'predictor': 'gpu_predictor',  ## Use GPU predictor
        'gpu_id': 0  ## GPU ID to use, we have one so it's not really a question
    }
    clf = XGBClassifier(**param)
    clf.fit(X_train_smote, y_train_smote)
    return roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

study = opt.create_study(direction='maximize')
study.optimize(objective, n_trials=200)
print(study.best_params)

[I 2024-05-08 14:31:00,610] A new study created in memory with name: no-name-c1070141-9fa2-4d9e-bf4a-886fc3c398f0
[I 2024-05-08 14:31:06,237] Trial 0 finished with value: 0.8627574147862004 and parameters: {'n_estimators': 1054, 'max_depth': 4, 'learning_rate': 0.03777726480583527, 'subsample': 0.5348060278931385, 'colsample_bytree': 0.8856755624380139, 'gamma': 1.2295596775306283, 'reg_alpha': 0.828279297328038, 'reg_lambda': 0.9578308745330025, 'min_child_weight': 5}. Best is trial 0 with value: 0.8627574147862004.
[I 2024-05-08 14:31:07,480] Trial 1 finished with value: 0.8595176487517379 and parameters: {'n_estimators': 671, 'max_depth': 4, 'learning_rate': 0.07303584355777529, 'subsample': 0.762638015252288, 'colsample_bytree': 0.8273910221436955, 'gamma': 8.82202618054061, 'reg_alpha': 0.6273449364779338, 'reg_lambda': 0.27728260085257994, 'min_child_weight': 7}. Best is trial 0 with value: 0.8627574147862004.
[I 2024-05-08 14:31:08,040] Trial 2 finished with value: 0.85587869827

{'n_estimators': 636, 'max_depth': 10, 'learning_rate': 0.03963044428484041, 'subsample': 0.8664709504304585, 'colsample_bytree': 0.548207422876625, 'gamma': 3.7734671914975255, 'reg_alpha': 0.8170272453855797, 'reg_lambda': 0.9031463458719654, 'min_child_weight': 4}


In [29]:
params_to_use = study.best_params
params_to_use['objective'] = 'binary:logistic'
params_to_use['n_jobs'] = -1
params_to_use['tree_method'] = 'gpu_hist'
params_to_use['predictor'] = 'gpu_predictor'
params_to_use['gpu_id'] = 0
clf = XGBClassifier(**params_to_use)
clf.fit(X_train_smote, y_train_smote)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.548207422876625, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None,
              gamma=3.7734671914975255, gpu_id=0, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.03963044428484041, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=10, max_leaves=0,
              min_child_weight=4, missing=nan, monotone_constraints='()',
              n_estimators=636, n_jobs=-1, num_parallel_tree=1,
              predictor='gpu_predictor', random_state=0,
              reg_alpha=0.8170272453855797, reg_lambda=0.9031463458719654, ...)

## Model Evaluation

In [30]:
def evaluate_model(model, inputs, true_vals):
    y_pred = model.predict(inputs)
    y_probs = model.predict_proba(inputs)[:, 1]
    roc_auc = roc_auc_score(true_vals, y_probs)
    f1 = f1_score(true_vals, y_pred)
    cm = confusion_matrix(true_vals, y_pred)
    precision = precision_score(true_vals, y_pred)
    recall = recall_score(true_vals, y_pred)
    accuracy = accuracy_score(true_vals, y_pred)
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:\n", cm)
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Accuracy: {accuracy:.4f}")

evaluate_model(clf, X_test, y_test)

ROC AUC Score: 0.8660
F1 Score: 0.3465
Confusion Matrix:
 [[25858   387]
 [ 1405   475]]
Precision: 0.5510
Recall: 0.2527
Accuracy: 0.9363


In [32]:
##Make the test set balanced, ie undersample the 0 target variable
mask_class_0 = (y_test == 0)
mask_class_1 = (y_test == 1)
X_test_class_0 = X_test[mask_class_0]
X_test_class_1 = X_test[mask_class_1]
y_test_class_0 = y_test[mask_class_0].to_numpy()
y_test_class_1 = y_test[mask_class_1].to_numpy()
n_samples_per_class = min(len(y_test_class_0), len(y_test_class_1))

##Take indeces at random and make a test set where the target variable is balanced
indices_class_0 = np.random.choice(range(len(y_test_class_0)), n_samples_per_class, replace=False)
indices_class_1 = np.random.choice(range(len(y_test_class_1)), n_samples_per_class, replace=False)
X_test_balanced = np.concatenate((X_test_class_0[indices_class_0], X_test_class_1[indices_class_1]), axis=0)
y_test_balanced = np.concatenate((y_test_class_0[indices_class_0], y_test_class_1[indices_class_1]), axis=0)

## Shuffle the balanced dataset
shuffled_indices = np.random.permutation(n_samples_per_class * 2)
X_test_balanced = X_test_balanced[shuffled_indices]
y_test_balanced = y_test_balanced[shuffled_indices]
evaluate_model(clf, X_test_balanced, y_test_balanced)

ROC AUC Score: 0.8650
F1 Score: 0.3992
Confusion Matrix:
 [[1855   25]
 [1405  475]]
Precision: 0.9500
Recall: 0.2527
Accuracy: 0.6197


## Model Explanation

In [34]:
shap.initjs()
X_test_df = pd.DataFrame(X_test, columns = X.columns)
explainer = shap.TreeExplainer(clf)

In [35]:
shap.initjs()
rand_start = np.random.randint(0, len(X_test)- 1000)
end = rand_start + 1000
shap.force_plot(
    explainer.expected_value, shap_values[rand_start:end, :], X_test_df.iloc[rand_start:end, :]
)

# Randomer Forest

In [ ]:
if window_user: df = pd.read_csv('/content/train.csv', index_col = 0) ##For windows user
else: df = pd.read_csv('/content/data/train.csv', index_col = 0)
df = df[df['age'] > 0] ##Remove the -1 observation
y = df['SeriousDlqin2yrs']
X = df.drop(columns = 'SeriousDlqin2yrs')
imp = IterativeImputer(max_iter=10, random_state=rand_val)
imp.fit(X)
X_imp = imp.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.25, random_state=rand_val, stratify = y)
smote = SMOTE(random_state=rand_val)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
param = {
        'projection_matrix': 'RerF',
        'n_estimators': 1000,
        'max_depth': 22,
        'min_samples_split': 50,
        'n_jobs': -1,
        'random_state': rand_val
        }
clf = rerfClassifier(**param)
clf.fit(X_train_smote, y_train_smote)

rerfClassifier(max_depth=12, min_samples_split=30, n_estimators=900, n_jobs=-1,
               random_state=42)

In [ ]:
def evaluate_model(model, inputs, true_vals):
    y_pred = model.predict(inputs)
    y_probs = model.predict_proba(inputs)[:, 1]
    roc_auc = roc_auc_score(true_vals, y_probs)
    f1 = f1_score(true_vals, y_pred)
    cm = confusion_matrix(true_vals, y_pred)
    precision = precision_score(true_vals, y_pred)
    recall = recall_score(true_vals, y_pred)
    accuracy = accuracy_score(true_vals, y_pred)
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:\n", cm)
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Accuracy: {accuracy:.4f}")

evaluate_model(clf, X_test, y_test)